

**Do not change this version from 0.45**



In [ ]:
pip install qiskit==0.45 qiskit-aer

INFO: pip is looking at multiple versions of qiskit-aer to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
import numpy as np, random
import math
import scipy
from matplotlib import pyplot as plt

# Importing standard Qiskit libraries
from qiskit import *
from qiskit import quantum_info
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info.operators import Operator
import qiskit.quantum_info as qi
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.primitives import Sampler as Sampler
from qiskit import Aer
from qiskit import execute,  BasicAer #execute
from qiskit.circuit.random import random_circuit
from qiskit import QuantumRegister
import numpy as np
from qiskit import ClassicalRegister
from qiskit.circuit.library import RYGate
from qiskit.quantum_info import DensityMatrix, partial_trace

#Basic Gates
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
I = np.identity(2)
H = 1/np.sqrt(2) * np.array([[1,1], [1,-1]])

#|0> and |1>
one = np.array([0,1])
zero = np.array([1,0])

def tensor(a,b,c):
  """
  Returns the tensor product of the arrays a, b, and c.

  Precondition: a, b, and c are numpy arrays
  """
  return np.kron(np.kron(a,b),c)

def tensorbig(lst):
    """
    Returns the tensor product of the arrays in lst.

    Preconditions:
    - lst contains only Numpy arrays
    """
    result = lst[0]
    for i in range(1, len(lst)):
        result = np.kron(result, lst[i])
    return result

def unitary(U):
  """
  Returns True if U is a unitary matrix.
  """
  return np.allclose(np.eye(U.shape[0]), np.dot(U, U.conjugate().T))


def flip(counts):
  """Given a dictionary counts with a string key, it returns
  a new dictionary with the key reversed and the value the same.

  Used for switching the endianness of the measurements."""
  res = {}
  for k,v in counts.items():
    res[k[::-1]] = v
  return res

In [ ]:
def numb(x):
  """Returns a length 8 vector where only index x is 1 and all other is 0"""
  return np.array([1 if x == i else 0 for i in range(8)])

#Swap between 1st and 3rd registers in 3 register system
s13 = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 1, 0],
                [0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1]])

#Swap between first two and second two registers
SWAP2 = np.dot(np.kron(s13, I), np.kron(I,s13))

#Swap between 1st and 4th registers in 4 register system
s14 = np.zeros((16,16))
s14[0,0] = s14[8,1] = s14[2,2] = s14[10,3] = s14[4,4] = s14[12,5] = s14[6,6] = s14[14,7] = s14[1, 8] = s14[9,9] = s14[3,10] = s14[11,11] = s14[5,12] = s14[13,13] = s14[7,14] = s14[15,15]  = 1

#Swap between first three and second three registers
SWAP3 = np.dot(tensor(s14, I,I), np.dot(tensor(I,s14,I), tensor(I,I,s14)))

#Basic swap
swap = .5 * (np.kron(X,X) + np.kron(I,I) + np.kron(Z,Z) + np.kron(Y,Y))
SWAP = swap

def eiswap(delta, neg = False, size = 1):
  """
  Returns an operator as a unitary matrix which implements e^iSWAPd.

  Parameter delta: the step size associated with the operation.
  Precondition: delta is a positive float

  Parameter neg: Whether to apply a factor of -1 in the exponent.
  Precondition: neg is a boolean

  Parameter: size: .5 * the number of qubits that will be used
  Precondition: size is an int between 1 and 3 inclusive
  """
  swap = .5 * (np.kron(X,X) + np.kron(I,I) + np.kron(Z,Z) + np.kron(Y,Y))
  if size == 1:
    sgate = swap
  elif size == 2:
    sgate = SWAP2
  elif size == 3:
    sgate = SWAP3
  phase = -1 if neg else 1
  return UnitaryGate(scipy.linalg.expm(-1j * phase * sgate * delta))

#Simulation Parameters
kappa = 1.8**2
kappa = np.sqrt(kappa)
gamma = .1
g = .1
wc = 0
we = [.18, .08]

EMITTERS = 2
assert EMITTERS <= len(we), "You need to define all the emitter frequencies"


zero2 = np.array([1,0,0,0])
one2 =  np.array([0,1,0,0])
two = np.array([0,0,1,0])
three = np.array([0,0,0,1])
a = (np.outer(zero2, one2) + np.sqrt(2) * np.outer(one2, two) + np.sqrt(3) * np.outer(two, three))


sigma = np.outer(zero, one)



#Interaction states
psis = []
psis.append(1/np.sqrt(2)*(numb(1)+numb(2)))
psis.append(1/np.sqrt(2)*(numb(1)-numb(2)))
psis.append(1/np.sqrt(2)*(numb(3)+numb(4)))
psis.append(1/np.sqrt(2)*(numb(3)-numb(4)))
psis.append(1/np.sqrt(2)*(numb(5)+numb(6)))
psis.append(1/np.sqrt(2)*(numb(5)-numb(6)))

#Pump states
ep = (kappa**2)/2
drive = []
drive.append(1/np.sqrt(2) * np.array([1, np.exp(-1j * wc * 100), 0, 0]))
drive.append(np.dot(np.kron(I, Z), drive[0]))
drive.append(1/np.sqrt(2) * np.array( [0, 1, np.exp(-1j * wc * 100), 0]))
drive.append(np.dot(np.kron(Z, I), drive[2]))
drive.append(1/np.sqrt(2) * np.array([0, 0, 1, np.exp(-1j * wc * 100)]))
drive.append(np.dot(np.kron(I, Z), drive[4]))

Ls = [kappa * a] + EMITTERS * [gamma * np.outer(zero, one)]
Hstates = [three, two, one2] + EMITTERS * [one] + EMITTERS * psis + 1 * drive
Hcoeffs = [3*wc, 2*wc, wc] + [we[i] for i in range(EMITTERS)] + EMITTERS * [g,-1*g, np.sqrt(2)*g, -1*np.sqrt(2) * g, np.sqrt(3) * g, -1*np.sqrt(3) * g] + [ep, -1*ep, ep * np.sqrt(2), -1 * np.sqrt(2) * ep, ep * np.sqrt(3), -1 * ep * np.sqrt(3)]


In [ ]:
#J-Matrices used in the fixed interaction
Jcav = np.block([[np.zeros((4, 4)), Ls[0].conjugate().T],
                   [Ls[0], np.zeros((4, 4))]])
Jemit = np.block([[np.zeros((2, 2)), Ls[1].conjugate().T],
                   [Ls[1], np.zeros((2, 2))]])

In [ ]:

def trace_cav(counts):
  """
  Given a counts dictionary of the full cavity-emitters system,
  returns a new dictionary for only the cavity counts.

  The cavity shold correspond to the first two indices of the key.
  """
  res = {}
  for key in counts:
    if key[:2] in res:
      res[key[:2]] += counts[key]
    else:
      res[key[:2]] = counts[key]
  return res

def trace_emit(counts, ind):
  """
  Given a counts dictionary of the full cavity-emitters system,
  returns a new dictionary for only the counts for one emitter.

  Parameter ind: the emitter index. Starts from 0.
  Precondition: ind is an int between 0 and emitters-1 inclusive.
  """
  res = {"0": 0, "1": 0}
  for key in counts:
    if key[2+ind] == "0":
      res["0"] += counts[key]
    else:
      res["1"] += counts[key]
  return res

def wml(start_photons = 1, t = .05, iters = 5, shots = 1000):
  """Runs a hybrid algorithm with J-Matrix fixed interation and DME Hamiltonian
  simulation.

  It sets up the quantum circuit and pumps the cavity to set the initial state.
  In each iteration, it first chooses a J-Matrix or a Hamiltonian program state.

  If a J-matrix is chosen, it implements the decay by applying the J-Matrix to
  the associated system and the auxiliary qubit. Then, the auxiliary is traced
  out.

  Otherwise, it decides which program state should be applied to the program
  register. Then, it applied e^iSWAPd to the system and program qubits.

  Finally, after all iterations are complete, it measures the system qubits
  and calculates the populations and g2 value.

  Parameter start_photons: The number of excitations the cavity starts with.
  Precondition: start_photons is 0, 1, 2, or 3.

  Parameter t: the simulation time.
  Precondition: t is a positive float.

  Parameter iters: the number of steps in the algorithm.
  Precondition: iters is a positive int.

  Parameter shots: the number of measurement shots to take.
  Precondition: shots is a positive int.
  """

  emitters = EMITTERS
  #Create a circuit
  anc = QuantumRegister(1, name="0")
  cav_init = QuantumRegister(2, name = 'cavity rho')
  emit_init = QuantumRegister(emitters, name = 'emitter rho')
  prog = QuantumRegister(3, name = "program")
  classical = ClassicalRegister(2 + emitters)
  circ = QuantumCircuit(anc, cav_init, emit_init, prog, classical)

  #Pump the cavity
  if start_photons > 1:
    circ.x(cav_init[0])
  if start_photons % 2 == 1:
    circ.x(cav_init[1])

  c = np.sum([np.linalg.norm(x)**2 for x in Ls]) + np.sum([np.abs(x) for x in Hcoeffs])
  HLprobs = [np.sum([np.linalg.norm(x)**2 for x in Ls]), np.sum([np.abs(x) for x in Hcoeffs])]/np.sum([np.sum([np.linalg.norm(x)**2 for x in Ls]), np.sum([np.abs(x) for x in Hcoeffs])])

  for iter in range(iters):
    if iter % 50000 == 0:
      print(iter)
    delta = c * t/iters
    sample = np.random.choice(["L", "H"], p = HLprobs)
    if sample == "L":
      lindblad = np.random.choice(range(emitters+1), p = [np.linalg.norm(x)**2 for x in Ls]/np.sum([np.linalg.norm(x)**2 for x in Ls]) )
      if lindblad == 0:
        circ.append(UnitaryGate(scipy.linalg.expm(-1j * Jcav * np.sqrt(c/c * t/(iters*HLprobs[1])))), reversed([anc[0]] + [cav_init[0], cav_init[1]]))
      else:
        circ.append(UnitaryGate(scipy.linalg.expm(-1j * Jemit * np.sqrt(c/c * t/(iters*HLprobs[1])))), reversed([anc] + [emit_init[lindblad-1]]))
      circ.reset(anc)
    else:
      hami = np.random.choice(range(len(Hcoeffs)), p = [np.abs(x) for x in Hcoeffs]/np.sum([np.abs(x) for x in Hcoeffs]))
      if hami < 3:
        circ.initialize(Hstates[hami], reversed(prog[:2]))
        circ.append(eiswap(delta, size = 2), reversed([cav_init[0], cav_init[1], prog[0], prog[1]]))
      elif hami in range(3, 3+emitters):
        circ.initialize(Hstates[hami], prog[0])
        circ.append(eiswap(delta), reversed([emit_init[hami-3], prog[0]]))
      elif hami < 3 + emitters + (emitters * 6):
        circ.initialize(Hstates[hami], reversed(prog[:3]))
        circ.append(eiswap(delta, size = 3, neg = Hcoeffs[hami] < 0), reversed([cav_init[0], cav_init[1], emit_init[(hami-(3+emitters)) // 6], prog[0], prog[1], prog[2]]))
      else:
          circ.initialize(Hstates[hami], reversed(prog[:2]))
          circ.append(eiswap(delta, size = 2, neg = Hcoeffs[hami] < 0), reversed([cav_init[0], cav_init[1], prog[0], prog[1]]))

    circ.reset(prog)

  circ.measure(range(1, 3 + emitters), classical)

  backend = Aer.get_backend("qasm_simulator")

  job = execute(circ, backend, shots=shots)
  result = job.result()
  counts = result.get_counts()
  counts = flip(counts)

  cav_counts = trace_cav(counts)
  population = (3 * cav_counts.get("11", 0) + 2 * cav_counts.get("10", 0) + 1 * cav_counts.get("01", 0))/shots
  emitvals = [trace_emit(counts, i).get("1")/shots for i in range(emitters)]
  try:
    g2 = (6 * cav_counts.get("11", 0) + 2 * cav_counts.get("10", 0))/(shots * population ** 2)
  except ZeroDivisionError:
    g2 = 0

  return g2, population, emitvals, cav_counts


print(wml(iters = 45, start_photons = 1, t = 1.3, shots = 600))


0
(0.6438688267022431, 1.8016666666666667, [0.56, 0.455], {'01': 169, '10': 285, '00': 32, '11': 114})


In [ ]:
def g2_experiment(t, shots):
  """
  Subroutine to calculate g2 values. Shots should be relatively high.

  Parameter t: the simulation time.
  Precondition: t is a positive float.

  Parameter shots: the number of shots to take.
  Precondition: shots is a positive int.
  """
  counts = wml(iters = 70000, start_photons = 1, t = t, shots = shots)[-1]
  population = (3 * counts.get("11", 0) + 2 * counts.get("10", 0) + 1 * counts.get("01", 0))/shots
  try:
    g2 = (6* counts.get("11", 0) + 2 * counts.get("10", 0))/(shots * (population ** 2))
  except ZeroDivisionError:
    g2 = 0
  return counts, g2

for _ in range(15):
  print(g2_experiment(1.5, 1500))

In [ ]:
def get_iters(t, c = 33.89):
  """Calculates the number of iterations needed for the simulation.

  This is based on the rule that ct/iters shuld be less than .001.

  Parameter t: the simulation time.
  Precondition: t is a positive float

  Parameter c: the weight of the operators.
  Precondition: c is a positive float.
  """
  return int(c * t/.001) + 1

get_iters(1.4)

47447